### import
- 기준일 다음날 정식,약식데이터 다운 > 약식 업로드 > 정식 양식 업로드 결과 (목록을 최대한 비슷하게 하기 위해 정식도 기준일 다음날 받는다.)
- 컷오프요일이 언제든 금액정보는 정확히 반영할 수 있음
- 유의사항
    - 진행상태가 실효라도 실효일이 없으면 실효되지 않을 수도 있다.(미납회수가 실효기준을 충족하고 입금예정일(실효예정일)에 입금이 없는 경우. 요일상관없이 3일뒤 실효확정되고, 그날 오후(아마 입금업무마간시간후겠지)에 실효일 찍히고 그 다음날 실효통지. 그런데 그전에 입금해버리면 실효안됨. 진행상태는 실시간이므로 약식통합이든, 정식만으로 하든 동일한 결과). 실효가 확정되기 전까지는 실효일자 없음. 따라서 실효일자 없는 실효는 컷오프일 기준으로는 엄밀히 말해 실효가 아니라 실효예정이다. -> 정상으로 변경
        - 완제일 없는 중도완제는 입금처리가 안 된 것일뿐 오후 되면 그날로 날짜 박힘.(중도완제 맞음)
    - 막 채권신고한 경우, 약식에서는 계좌번호 나오나, 정식에서는 접수통지중으로 나오는 경우, 약식을 기준으로 병합하므로 진행중으로 처리된다. 대신 정식에만 있는 칼럼은 모두 빈값

#### 변경로그
- 실효예정 : [포함,실효but실효일없음] > 정상이행으로 240823

In [15]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
import functions
from datetime import datetime, timedelta

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

### 파일 읽기

In [16]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
약식병합 = True ############# 금요일(일요일)이 컷오프가 아니라면 약식병합해야-------
basedate = "241031" # 정식 업로드결과 파일 날짜 컷오프일에 맞춰 수정
company = "대성"  # 통합 or 대성, 통합=솔림+신탁88
basedate_계좌 = "241031" # "yymmdd" 또는 basedate ######################## 계좌조회새창 받기 전이면 전달로
########################################################

# 읽을 파일명
filename_read = basedate + "_계좌별 진행상황 조회_" + company

# 파일 읽기
if 약식병합 : 
    suffix_정식 = "_업로드결과.xlsx"
    suffix_약식 = "_약식_업로드결과.xlsx"

    # 계좌별 진행상황 조회 파일 읽기
    정식_ori = pd.read_excel(join(path_base, filename_read + suffix_정식 ), dtype=nauri_dtype).fillna("")
    약식_ori = pd.read_excel(join(path_base, filename_read + suffix_약식 ), dtype=nauri_dtype).fillna("")
else : 
    suffix_정식 = "_업로드결과.xlsx"
    정식_ori = pd.read_excel(join(path_base, filename_read + suffix_정식 ), dtype=nauri_dtype).fillna("")
    
# 계좌 (메모 : 통합신고건 처리위해) --------------
if company == "통합" : 
    if basedate_계좌 != basedate : # 전 달 거 읽는다면 pkl 파일
        dir_account = join(r"D:\3.자산\전산 dataset", "솔림", basedate_계좌, "pkl")
    else :
        dir_account = join(r"D:\3.자산\전산 dataset", "솔림", basedate_계좌)
else : 
    if basedate_계좌 != basedate : # 전 달 거 읽는다면 pkl 파일
        dir_account = join(r"D:\3.자산\전산 dataset", "대성", basedate_계좌, "pkl")
    else :
        dir_account = join(r"D:\3.자산\전산 dataset", "대성", basedate_계좌)
file_list = os.listdir(dir_account)
account_filename = [file for file in file_list if file.startswith("계좌조회새창")][0]

if basedate_계좌 != basedate : 
    account_ori = pd.read_pickle(join(dir_account, account_filename))[["계좌키","채무자키","메모"]].fillna("")
else :
    account_ori = pd.read_excel(join(dir_account, account_filename), dtype=account_dtype)[["계좌키","채무자키","메모"]].fillna("")

### 전처리

##### 키 중복건 처리

In [17]:
# 재시작 위해 복사본 만들기
if 약식병합 : 
    약식 = 약식_ori.copy()
    print("약식건수", len(약식))
    # 키값에 중복건 있는지 확인, merge때문에 약식, 정식 모두 해야함
    약식["pk"] = 약식.신청인명 + 약식.신청인주민번호 + 약식.접수번호 + 약식.계좌번호
    print("약식 키 중복건")
    display(약식[약식.pk.duplicated(keep=False)])

정식 = 정식_ori.copy()
print("정식건수", len(정식))

정식["pk"] = 정식.신청인명 + 정식.신청인주민번호 + 정식.접수번호 + 정식.계좌번호
print("정식 키 중복건")
display(정식[정식.pk.duplicated(keep=False)])

account = account_ori.copy()

약식건수 991
약식 키 중복건


,채무자키,계좌키,보증인키,신용회복키,채무자명,채무상태,현재채권구분,신청인명,신청인주민번호,접수번호,...,등록파일,수정파일,삭제여부,등록일자,등록유저,수정일자,수정유저,확인결과,저장결과,pk


정식건수 987
정식 키 중복건


,채무자키,계좌키,보증인키,신용회복키,채무자명,채무상태,현재채권구분,신청인명,신청인주민번호,접수번호,...,등록파일,수정파일,삭제여부,등록일자,등록유저,수정일자,수정유저,확인결과,저장결과,pk


In [18]:
# 중복건 확인 후 삭제 : 솔림, 신탁 중복 신고건일 것이다.
####################################################################
정식_삭제전 = len(정식)
정식 = 정식.drop_duplicates(subset=["pk"], keep='first', inplace=False, ignore_index=True)
정식_삭제후 = len(정식)
if 약식병합 : 
    약식_삭제전 = len(약식)
    약식 = 약식.drop_duplicates(subset=["pk"], keep='first', inplace=False, ignore_index=True)
    약식_삭제후 = len(약식)
print(f'정식 삭제건수 : {정식_삭제전} - {정식_삭제후} = {정식_삭제전-정식_삭제후}')
print(f'약식 삭제건수 : {약식_삭제전} - {약식_삭제후} = {약식_삭제전-약식_삭제후}')

정식 삭제건수 : 987 - 987 = 0
약식 삭제건수 : 991 - 991 = 0


In [19]:
# 포함 & 실효 but 실효일 없음 -> 정상이행으로(실효예정일뿐)
# (기준일 오후 5:30 이후에 받은 자료일 때만 정확.)
if 약식병합 : 
    실효임박index = 약식[(약식["계좌진행상태내용"]=="채무조정포함")&(약식["신청인진행상태"]=="중도실효")&(약식["실효/완제/합의서포기일자"]=="")].index
    약식.loc[실효임박index, "신청인진행상태"] = "정상이행"
else : 
    실효임박index = 정식[(정식["계좌진행상태내용"]=="채무조정포함")&(정식["신청인진행상태"]=="중도실효")&(정식["실효/완제/합의서포기일자"]=="")].index
    정식.loc[실효임박index, "신청인진행상태"] = "정상이행"

In [20]:
if 약식병합 : 
        # 정식 : 불필요 칼럼 삭제
        키열 = ["신청인명", "신청인주민번호", "접수번호", "계좌번호"]

        나우리칼럼불요 = ["채무자명", "채무상태", "현재채권구분","신청인진행상태변경전", "특별상환유예", "pk", # pk 추가
                "변경일", "변경구분", "신용회복진행상태","신복아이디","등록파일","수정파일","삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"]
        나우리칼럼필요 = ["채무자키", "계좌키", "보증인키", "신용회복키"]
        약식열 = ["거래일","심의차수","신청인진행상태","계좌진행상태내용","실효/완제/합의서포기일자","접수통지일","접수일","점포명","확정일","합의서체결일","상환개시일",
                "최초대출원금","조정전원금","조정전이자","조정전연체이자","조정전비용","조정전합계","조정후원금","조정후이자","조정후연체이자","조정후비용","조정후합계",
                "원금감면여부","조정대상포함여부","총상환기간","유예기간","원금균등상환기간","원리균등상환기간","이자상환기간","납입회차","연체기간","총납입금액","총납입원금",
                "총납입이자","총납입기타채무","상환후잔액","감면율","급여가압류원금","채무구분","주채무자주민번호","주채무자성명","동의요청일","동의확정일","재조정여부","재조정횟수",
                "수정조정여부","수정조정횟수","재조정접수일자","재조정처리일자","재조정반송일자","실효취소요청일자","실효원상회복일자","상각구분","신청구분","일시납감면율",
                "차상위계층여부","미납발생일","부동산담보대출여부","거치기간","감면방식","상환방식","담보권실행유예기간","특별면책","재난상환유예","특례지원","이자율재조정","거치이자납입유예"]

        정식.drop(columns= 나우리칼럼불요 + 나우리칼럼필요 + 약식열, inplace=True, axis=1)

        # 약식 : 신복 약식에 있는 칼럼만 남기기
        약식 = 약식.loc[:, 나우리칼럼필요 + 키열 + 약식열]
        
        # merge : 약식을 기준으로 정식에만 있는 칼럼 merge(약식에도 있는 칼럼은 drop했으므로 그대로 붙이면 됨)
        raw = pd.merge(약식, 정식, on=["신청인명", "신청인주민번호", "접수번호", "계좌번호"], how='left')
        print(len(raw), ", 병합전 약식 개수 : ", len(약식))
        
else : 
        raw = 정식

991 , 병합전 약식 개수 :  991


In [21]:
# 신용회복 자료와 칼럼 순서 맞추기 : 재난상환유예를 특별면책 뒤, 특례지원 앞으로
cols = ["채무자키","계좌키","보증인키","신용회복키","신청인명","신청인주민번호","접수번호","거래일","계좌번호","심의차수","신청인진행상태","계좌진행상태내용","실효/완제/합의서포기일자",
        "접수통지일","접수일","점포명","확정일","합의서체결일","상환개시일","대출과목","조정전이율","조정후이율","최초대출원금","조정전원금","조정전이자","조정전연체이자","조정전비용",
        "조정전합계","조정후원금","조정후이자","조정후연체이자","조정후비용","조정후합계","원금감면여부","조정대상포함여부","총상환기간","유예기간","원금균등상환기간","원리균등상환기간",
        "이자상환기간","납입회차","연체기간","총납입금액","원금균등시작회차","원금균등종료회차","원리균등시작회차","원리균등종료회차","이자상환시작회차","이자상환종료회차","원금균등채무액",
        "총납입원금","총납입이자","총납입기타채무","상환후잔액","감면율","추심원금","급여가압류원금","채무구분","주채무자주민번호","주채무자성명","시작회차1","종료회차1","채무조정1적용이율1",
        "채무조정2적용이율1","채무조정1월상환액1","채무조정2월상환액1","시작회차2","종료회차2","채무조정1적용이율2","채무조정2적용이율2","채무조정1월상환액2","채무조정2월상환액2","시작회차3",
        "종료회차3","채무조정1적용이율3","채무조정2적용이율3","채무조정1월상환액3","채무조정2월상환액3","시작회차4","종료회차4","채무조정1적용이율4","채무조정2적용이율4","채무조정1월상환액4",
        "채무조정2월상환액4","시작회차5","종료회차5","채무조정1적용이율5","채무조정2적용이율5","채무조정1월상환액5","채무조정2월상환액5","시작회차6","종료회차6","채무조정1적용이율6","채무조정2적용이율6",
        "채무조정1월상환액6","채무조정2월상환액6","시작회차7","종료회차7","채무조정1적용이율7","채무조정2적용이율7","채무조정1월상환액7","채무조정2월상환액7","시작회차8","종료회차8","채무조정1적용이율8",
        "채무조정2적용이율8","채무조정1월상환액8","채무조정2월상환액8","시작회차9","종료회차9","채무조정1적용이율9","채무조정2적용이율9","채무조정1월상환액9","채무조정2월상환액9","시작회차10","종료회차10",
        "채무조정1적용이율10","채무조정2적용이율10","채무조정1월상환액10","채무조정2월상환액10","동의요청일","동의확정일","재조정여부","재조정횟수","수정조정여부","수정조정횟수","재조정이전납입금액",
        "원금완제면제이자","출금정지등록여부","재조정접수일자","재조정처리일자","재조정반송일자","실효취소요청일자","실효원상회복일자","상각구분","신청구분","일시납감면율","차상위계층여부","미납발생일",
        "부동산담보대출여부","거치기간","감면방식","상환방식","체증식계좌여부","체증전구간의시작회차","체증전구간의종료회차","체증전구간의기준기간","변제금수취계좌","담보권실행유예기간","특별면책",
        "재난상환유예","특례지원","이자율재조정","거치이자납입유예"
]
raw = raw.loc[:,cols]


### 반영OPB 등

In [22]:
# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 4
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

# 전산미등록건 처리 #####################################################
if company == "통합" : 
    미등록주민번호 = ['771027-2221231'] #강민경
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
else :  # 대성
    미등록주민번호 = ['741008-1037838'] #강승욱
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
########################################################################

In [23]:
# 주민 계좌 중복건 : 전산에는 중복건 있더라도 신복위에서 받은 데이터에는 없는것이 당연(솔림,신탁 동시 채권신고요청시에는 있을 수 있음)
raw[raw.duplicated(subset=["신청인주민번호", "계좌번호"], keep=False)]

,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,접수번호,...,체증전구간의시작회차,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,재난상환유예,특례지원,이자율재조정,거치이자납입유예


In [24]:
# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호
r = raw.재조정여부

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
    (r == "Y")
]
values = ["확정", "채무조정제외","완제","채권신고전","실효","확정"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위  
# 정상건인데 상환후잔액이 없는 경우는, 실효취소 및 재조정합의 후 첫회차 입금이 안 된 경우
# 조정후비용이나 이자가 있는 경우가 있기 때문에 원래 2순위로 조정후 합계가 반영되어야 하나, 회계법인 평가법에 아래와 같이 나와 있고, 이를 다른 회사도 적용하고 있다고 해서 그냥 이대로 적용하기로.
# 또한 약식을 활용하면 상환후잔액이 없는 경우가 거의 없어서 큰 문제는 없음.
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금 + raw.조정전비용   # 조정정비용 추가 2024-04-03

# 미확정-조정전원금0 은 이자채권으로 그냥 0이 맞다.
conds = [(x == "미확정"), (x =="확정")&(a!=0), (x =="확정")&(b!=0), (x =="확정")&(c!=0), (x =="확정")&(d!=0)]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = np.nan

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.상환방식

conds = [x.str.contains("원금균등"), x.str.contains("원리금균등")] #체증식(원금균등)이란게 있어서 contains로 바꿈

# 조건에 따라 값을 설정
values = ["개인","프리"]

# 기본값 설정
default_value = "" # 미확정

# 반영OPB 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

### 시트작성 및 체크 : 최종,정상,제외

In [25]:
# 최종, 확정미확정, 일부계좌제외 시트
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
확정미확정 = raw.query('진행구분 in ["확정", "미확정"]').copy()
일부계좌제외 = raw.query('진행구분 == "채무조정제외" and 채무자키 != ""') # 계좌번호 변경한 건들 있어서 채무자키 없을 수 있음, 20240117 기준 솔림 8건
# 일부계좌제외는 자산정리 채무상태에서 신복으로 처리하여 오류탐지 제외 ~~

# 전산미등록건, 일부계좌제외
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)
확정미확정.drop(확정미확정[확정미확정.채무자키=='전산미등록건'].index, inplace=True)
일부계좌제외.drop(일부계좌제외[일부계좌제외.채무자키=='전산미등록건'].index, inplace=True)

# 포함과 제외가 공존하는 제외건만 : 제외자 주민번호가 확정미확정에도 있으면
일부계좌제외 = 일부계좌제외.merge(확정미확정["신청인주민번호"], how="inner", on=["신청인주민번호"])

# 순번열 넣기
if "순번" not in raw.columns : 
    raw.insert(0, "순번", range(1,len(raw)+1))
    최종.insert(0, "순번", range(1,len(최종)+1))
    확정미확정.insert(0, "순번", range(1,len(확정미확정)+1))
    일부계좌제외.insert(0, "순번", range(1,len(일부계좌제외)+1))

In [26]:
# 체크
# 반영OPB
반영OPB오류 = raw[(raw.반영OPB == np.nan)|(raw.반영OPB == 0)|(raw.반영OPB=="")]
cols = ["채무자키", "계좌키", "진행구분", "상환구분","반영OPB", "조정전원금"]
키cols = ["채무자키", "계좌키", "보증인키", "신용회복키", "진행구분", "반영OPB", "상환구분", "신청인명", "신청인주민번호", "신청인진행상태", "계좌진행상태내용", "채무구분"]
if len(반영OPB오류) > 0 : 
    print('반영OPB 0원 ------')
    print('원금,법비용 0원계좌라도 상환예정액 조금이라도 나옴. 미확정건이면 전산opb를 따르니 0원 되겠지') ######################
    display(반영OPB오류[cols])
else : print("반영OPB 오류 없음")

# 확정미확정에 신용회복키 중복여부
신용회복키중복 = 확정미확정[확정미확정.duplicated('신용회복키', keep=False)]
if len(신용회복키중복) > 0 : 
    print('신용회복키 중복건 ------ 신용회복키 빈값 없애려면 업로드 후에 재다운로드해야')
    display(신용회복키중복[키cols])
else : print("확정미확정에 신용회복키 중복 없음")

# 채무자키, 계좌키, 신용회복키 매칭 오류
최종키오류 = 최종.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
확정미확정키오류 = 확정미확정.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
print(f"전체 : {len(raw)}, 최종 : {len(최종)}, 확정미확정 : {len(확정미확정)}")
print(f"최종에서 키 없는 행 수 : {len(최종키오류)}, 확정미확정에서 키없는 행 수 : {len(확정미확정키오류)}")
if len(확정미확정키오류)>0 : 
    print('확정미확정 채무자키, 계좌키, 신복키 중 하나라도 없는 경우 ------')
    display(확정미확정키오류[키cols])
else : print("확정미확정 건에 키매칭 실패 없음")

# 확정 건 중 상환구분 없는 값
print("확정 건 중 상환구분 없는 값 :", len(확정미확정.query('상환방식=="확정" and 상환구분==""')))

# 일부계좌제외에 보증채무있는지
print("일부계좌제외에 보증채무수 :", len(일부계좌제외.query('채무구분=="보증채무"'))) # 있으면 드랍할지 말지 그때 결정

반영OPB 오류 없음
확정미확정에 신용회복키 중복 없음
전체 : 991, 최종 : 960, 확정미확정 : 670
최종에서 키 없는 행 수 : 9, 확정미확정에서 키없는 행 수 : 0
확정미확정 건에 키매칭 실패 없음
확정 건 중 상환구분 없는 값 : 0
일부계좌제외에 보증채무수 : 0


In [ ]:
# # # 키값 없는 거 수정
# 확정미확정.loc[1870,["신용회복키"]] = "26306352"

### 통합신고건 처리하기

In [27]:
통합신고계좌 = account.query("메모.str.contains('신용회복 통합신고')")
print("통합신고계좌수 :",len(통합신고계좌))
print("추가전 확정미확정 건수 :", len(확정미확정))

cnt = 확정미확정.순번.iloc[-1]

for i, v in 통합신고계좌.iterrows() : 
    피통합계좌 = re.search(r"[0-9]{9,9}", v.메모)[0]
    temp = 확정미확정.query('계좌키 == @피통합계좌').copy()
    if len(temp) == 1 :
        cnt += 1
        temp['순번'] = cnt
        temp['채무자키'] = v.채무자키
        temp['계좌키'] = v.계좌키
        temp[['신용회복키','계좌진행상태내용']] = '통합신고 ' + v.계좌키
        temp['반영OPB'] = 0
        
        # 행추가
        확정미확정 = pd.concat([확정미확정, temp])
    elif len(temp) == 0 :
        print("통합신고 취소, 원상회복 분할적용 확인 : ", v.계좌키, v.채무자키)
    
    else : 
        print("중복계좌키 발견 : ", v.계좌키, v.채무자키)
        
print("추가후 확정미확정 건수 :", len(확정미확정))

전체요약 = 확정미확정.groupby('진행구분').agg({'신용회복키':'count', '반영OPB':'sum'})
전체요약.loc["합계", :] = 전체요약.sum()
display(전체요약.applymap('{:,.0f}'.format))

통합신고계좌수 : 0
추가전 확정미확정 건수 : 670
추가후 확정미확정 건수 : 670


,신용회복키,반영OPB
진행구분,,
미확정,6,"10,511,441"
확정,664,"674,724,062"
합계,670,"685,235,503"


### 파일 저장

In [28]:
# 저장
# 기준데이터 저장 경로
if 약식병합 : 
    filename_write = basedate+"_신용회복_기준데이터_약식병합_"+ ("대성" if company=="대성" else "솔림")
else : 
    filename_write = basedate+"_신용회복_기준데이터_"+ ("대성" if company=="대성" else "솔림")
    
fullpath_result = join(os.path.dirname(path_base),"기준데이터", filename_write +".xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    raw.to_excel(writer, sheet_name='원본', index=False)
    최종.to_excel(writer, sheet_name='최종', index=False)
    확정미확정.to_excel(writer, sheet_name='확정,미확정', index=False)
    일부계좌제외.to_excel(writer, sheet_name='일부계좌제외', index=False)

### 채권등급판별 : 변제율
- 진행상태 로그도 받아서 변경전 및 변경후에서 실효이력 있는 거 체크
- 당사전기간 입금건도 체크하는 게 좋겠다.
- 납입회차말고 현재회차 등도 추가

In [ ]:
# # 매입가 파일 갱신된 경우 실해
# 매입가 = pd.read_excel(r"D:\3.자산\매입가.xlsx", dtype={"채무자키":str, "계좌키":str, "계좌번호":str})
# 매입가.to_pickle(r"D:\3.자산\매입가.pkl")

#### 파일읽기
- 접수번호로그검색용 채무자키 20410653

In [ ]:

# 계좌별 진행상황 파일 읽기
########################################################
basedate = "240930" 
company = "솔림"
pkl = True # 신복기준데이터, 계좌만 상관있음
########################################################
path_base = join(r"D:\3.자산\전산 dataset", company, basedate)
if pkl :
    path_base_pkl = join(path_base, "pkl")
else : 
    path_base_pkl = path_base
    
dtype = {"채무자키":str, "계좌키":str, '신용회복키' : str}# 시간정보 무시하고 날짜만 읽기 cf 'datetime64[ns]'
# 파일읽기 : 신복기준데이터, 계좌, 접수번호로그, 합의서체결일로그, 매입가
    # 신복기준데이터
if pkl : 
    credits_ori = pd.read_pickle(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터"))
else : 
    credits_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터"), sheet_name="확정,미확정", dtype=nauri_dtype).fillna("")

    # 계좌
# account = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\240826\_계좌~~.xlsx", dtype={'채무자키':str, '계좌키':str, '타채무자키':str})[["계좌키", "채권구분", "OPB", "컷오프일", "담보제공처"]]
if pkl : 
    account = pd.read_pickle(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="계좌조회새창"))[["계좌키", "채권구분", "OPB","컷오프일", "담보제공처", "최초연체일"]]
else : 
    account = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="계좌조회새창"), dtype=dtype)[["계좌키", "채권구분", "OPB","컷오프일", "담보제공처", "최초연체일"]]

account.rename(columns={"OPB":"최초OPB", "담보제공처":"풀"}, inplace=True)
account["풀"] = account["풀"].str.split("_").str.get(0).fillna("")

    #### 신용회복키 포함 다운 ####
    # 접수번호로그 : 재접수건만 나옴. 접수'일'로그는 재접수아닌 것도 나옴.
접수번호로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="접수번호로그"), dtype=dtype)[["신용회복키"]]
        # 재접수 아닌 건 삭제 : 신용회복키 고유
접수번호로그_ori = 접수번호로그_ori[접수번호로그_ori.duplicated('신용회복키', keep=False)]
접수번호로그_ori["재접수횟수"] = 접수번호로그_ori.groupby('신용회복키')['신용회복키'].transform('count') - 1
        # 신용회복키 중복값제거
접수번호로그_ori = 접수번호로그_ori.drop_duplicates('신용회복키', keep='first')

    # 합의서체결일로그
합의서체결일로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="합의서체결일로그"), dtype=dtype)[["신용회복키","변경후합의서체결일","변경일"]]
합의서체결일로그_ori["변경일"] = pd.to_datetime(합의서체결일로그_ori["변경일"], errors='coerce')
합의서체결일로그_ori = 합의서체결일로그_ori.rename(columns={'변경후합의서체결일':'합의서체결일로그'})
합의서체결일로그_ori["합의서체결일로그"] = pd.to_datetime(합의서체결일로그_ori["합의서체결일로그"], errors='coerce')

    # 감면율로그
감면율로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="감면율로그"), dtype=dtype)[["신용회복키","변경후감면율", "변경일"]]
감면율로그_ori["변경일"] = pd.to_datetime(감면율로그_ori["변경일"], errors='coerce')

    # 신청인진행상태로그 : 실효여부체크용, 변경후는 변경전을 모두 포함(변경후에만 있는 것은 최종실효건). 신복키 반복 횟수는 실효횟수
진행상태로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="신청인진행상태로그"), dtype=dtype)[["신용회복키","변경후신청인진행상태","변경일"]]
진행상태로그_ori = 진행상태로그_ori.rename(columns={'변경후신청인진행상태':'진행상태로그'})
        # 상태값 실효인 것만 남기기
진행상태로그_ori = 진행상태로그_ori[진행상태로그_ori["진행상태로그"].str.contains('실효')]
진행상태로그_ori["변경일"] = pd.to_datetime(진행상태로그_ori["변경일"], errors='coerce')
진행상태로그_ori["실효횟수"] = 진행상태로그_ori.groupby('신용회복키')['신용회복키'].transform('count')
        # 중복값제거
진행상태로그_ori = 진행상태로그_ori.drop_duplicates('신용회복키', keep='first')

    # 상환후잔액로그 : 조정후원금과 차이나는 경우가 있어서(재조정일과 컷오프일이 차이가 날수록) 이왕이면 용량이 커도 상환후잔액으로
상환후잔액로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="상환후잔액로그"), dtype=dtype)[["신용회복키","변경후상환후잔액"]]
        # 최초건만 남기기
상환후잔액로그_ori = 상환후잔액로그_ori.drop_duplicates('신용회복키', keep='first')
매입후최초상환후잔액_ori = 상환후잔액로그_ori.rename(columns={'변경후상환후잔액':'매입후최초상환후잔액'})

    # 매입가
매입가 = pd.read_pickle(r"D:\3.자산\매입가 정리\매입가.pkl")
매입가.dropna(inplace=True)

    # 입금 : 기본칼럼("계좌키", "입금일", "입금합계", "입금자구분","입금구분"), 최근 6개월 / 전기간
입금_ori = pd.read_excel(functions.키워드로파일명찾기(path_base, "신용회복_입금조회새창"), dtype=dtype)[["계좌키", "입금일", "입금합계", "입금자구분","입금구분"]]
입금_ori["입금일"] = pd.to_datetime(입금_ori["입금일"], errors='coerce')
입금_ori = 입금_ori[입금_ori["입금구분"]=="신용회복"]
입금_ori = 입금_ori.drop(columns=["입금구분"])
입금_ori["입금자구분"] = 입금_ori["입금자구분"].replace({"채무자":"주채무", "보증인":"보증채무"})

In [ ]:
# # pkl 저장
# credits_ori.to_pickle(join(path_base_pkl, os.path.splitext(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터", 전체경로=False))[0]+".pkl"))
# account.to_pickle(join(path_base_pkl, os.path.splitext(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="계좌조회새창", 전체경로=False))[0]+".pkl"))

In [ ]:
credits = credits_ori.copy()
접수번호로그 = 접수번호로그_ori.copy()
합의서체결일로그 = 합의서체결일로그_ori.copy()
감면율로그 = 감면율로그_ori.copy()
진행상태로그 = 진행상태로그_ori.copy()
매입후최초상환후잔액 = 매입후최초상환후잔액_ori.copy()

입금_전기간 = 입금_ori.copy()
# 6개월
basedate_dt = datetime.strptime(basedate,'%y%m%d')
start_date = basedate_dt - pd.DateOffset(months=6)
입금_6개월 = 입금_ori[(start_date < 입금_ori["입금일"]) & (입금_ori["입금일"] <= basedate_dt)].copy()

#### 파일병합

In [ ]:
print(len(credits))
# 계좌와 병합
credits = credits.merge(account, how='left')
# 풀명칭 변경
credits["풀"] = credits["풀"].replace({'P1': 'P01', 'P2': 'P02', 'P3': 'P03', 'P4': 'P04', 'P5': 'P05', 'P6': 'P06', 'P7': 'P07', 'P8': 'P08', 'P9': 'P09'})
print("계좌병합", len(credits))

# 날짜칼럼 타입변경
credits["컷오프일"] = pd.to_datetime(credits["컷오프일"], errors='coerce')
credits["접수일"] = pd.to_datetime(credits["접수일"], errors='coerce')
credits["상환개시일"] = pd.to_datetime(credits["상환개시일"], errors='coerce')
credits["합의서체결일"] = pd.to_datetime(credits["합의서체결일"], errors='coerce')
credits["최초연체일"] = pd.to_datetime(credits["최초연체일"], errors='coerce')

# 접수번호로그와 병합
credits = credits.merge(접수번호로그, how='left')
credits["재접수횟수"] = credits["재접수횟수"].replace(np.nan,0).astype(int)

print("접수번호", len(credits))



# 합의서체결일로그 : 재조정건 최초합의서체결일 작성
합의서체결일로그 = 합의서체결일로그.merge(credits[["신용회복키", "접수일"]], how='left')
    # 재접수 이전 건은 삭제 : 현재 접수일보다 합의서체결일이 나중인 것, na유의
합의서체결일로그 = 합의서체결일로그[(합의서체결일로그['합의서체결일로그'] > 합의서체결일로그['접수일']) | 합의서체결일로그['접수일'].isna()]
    # 가장 빠른 합의서체결일 남기기
합의서체결일로그 = 합의서체결일로그.drop_duplicates(subset=["신용회복키"], keep='first')
합의서체결일로그 = 합의서체결일로그[["신용회복키", "합의서체결일로그","변경일"]].rename(columns={"합의서체결일로그": "최초합의서체결일"})
    # 병합 : 재조정안한 것 중 합의서체결일과 최초합의서체결일이 다른 경우는 최근에 합의서체결돼서 한곳에는 반영이 안 된 경우뿐
credits = credits.merge(합의서체결일로그[["신용회복키", "최초합의서체결일"]], how='left') # 변경일은 감면율로그와 병합시 사용

print("합의서체결일", len(credits))


# 감면율로그
    # 현재 접수건 중 최초감면율만 남기기 : 최초 합의서체결일 직전
감면율로그 = 감면율로그.merge(합의서체결일로그, on=["신용회복키"], how='left', suffixes=["_감면", "_합의"]) # 최초합의서체결일 na값은 최근 접수건에선 확정전(실효), 확정건이 아니므로 필요없음
감면율로그["날짜차이"] = (감면율로그["변경일_감면"] - 감면율로그["변경일_합의"]).abs() 
감면율로그 = 감면율로그.dropna(subset=["날짜차이"])
감면율로그 = 감면율로그.loc[감면율로그.groupby('신용회복키')['날짜차이'].idxmin()] # inxmin : 최소값을 가지는 index반환
감면율로그 = 감면율로그.rename(columns={"변경후감면율":"최초감면율"})

    # 병합
credits = credits.merge(감면율로그[["신용회복키", "최초감면율"]], how='left')
    # 감면율 변동 없었던 것은 최초감면율로 : 뒤에 연산과정 단순화
credits.loc[credits["최초감면율"].isna(), "최초감면율"] = credits["감면율"]
    # 감면율 변동없었던 것
감면율로그_unique = 감면율로그_ori[~감면율로그_ori.duplicated('신용회복키',keep=False)]

print("감면율", len(credits))


# 진행상태로그 병합
credits = credits.merge(진행상태로그[["신용회복키","실효횟수"]], how='left')

print("진행상태", len(credits))

# 매입후최초상환후잔액 병합
credits = credits.merge(매입후최초상환후잔액[["신용회복키","매입후최초상환후잔액"]], how='left')
print("매입후최초상환후잔액", len(credits))

# 매입가 병합
credits = credits.merge(매입가, how='left') # 매입가 없는 거 있음
print("매입가", len(credits))

# 입금_전기간
입금합계_전기간 = 입금_전기간.groupby(["계좌키", "입금자구분"]).sum('입금합계').reset_index()
입금합계_전기간 = 입금합계_전기간.rename(columns={"입금자구분":"채무구분", "입금합계":"입금합계_전기간"})
# 중복값제거
입금합계_전기간 = 입금합계_전기간.drop_duplicates(["계좌키", "채무구분"], keep='first')
# 병합
credits = credits.merge(입금합계_전기간, on=["계좌키", "채무구분"], how='left')
print("입금합계_전기간", len(credits))

# 입금_6개월
입금합계_6개월 = 입금_6개월.groupby(["계좌키", "입금자구분"]).sum('입금합계').reset_index()
입금합계_6개월 = 입금합계_6개월.rename(columns={"입금자구분":"채무구분", "입금합계":"입금합계_6개월"})
# 중복값제거
입금합계_6개월 = 입금합계_6개월.drop_duplicates(["계좌키", "채무구분"], keep='first')
# 병합
credits = credits.merge(입금합계_6개월, on=["계좌키", "채무구분"], how='left')
print("입금합계_6개월", len(credits))


#### 연산 및 저장

In [ ]:
# 재조정이전납금금액(조정금액만, 프리이자x) 및 총납입금액은 모두 현재 채권자에게 입금된 금액만을 대상으로 한다!!!

# 전처리
        # 진행구분 세분화
ind = credits[(credits.진행구분=="미확정")&(credits.재조정여부=="Y")].index
credits.loc[ind, "진행구분"] = "재조정중"

        # "", na, 형변환
credits["재조정이전납입금액"] = credits["재조정이전납입금액"].replace('', 0).astype(int)
# credits["재조정이전납입금액"] = credits["재조정이전납입금액"].astype(int)

# 1-1) 최초 조정후원금
        # 당사 최초 조정후원금(상환후잔액) : 매입 후 남은 조정후원금, 금액조정된 경우 차이 있을 수 있음
credits["접수후최초조정후원금"] = np.where(
    ((credits["조정후원금"] == 0)&(credits["조정전원금"] != 0)&(credits["진행구분"] == "확정")), 
    credits["재조정이전납입금액"] + credits["총납입원금"],  # 조정후원금이 0일 때
    credits["재조정이전납입금액"] + credits["조정후원금"]  # 그렇지 않을 때
)
ind = credits[(credits["상환구분"]=="프리")|(credits["원금감면여부"]=="N")|(credits["접수후최초조정후원금"]>credits["조정전원금"])].index
credits.loc[ind, ["접수후최초조정후원금"]] = credits["조정전원금"] # 접수후최초조정후원금 != 조정전원금 왜냐면 매입사 변제


#         # 매입전 최초 조정후원금 : 감면율이 반올림되어있어 약간 오차 있을 수 있음  <---- 재조정이전납입금액이 매입전은 포함하지 않은 줄 알았을 때 작성한 코드
# 감면율 : (조정후합계-비용) / (조정전합계-비용)
# 비용제외조정전합계 = credits["조정전원금"] + credits["조정전이자"] + credits["조정전연체이자"]
# credits["접수최초조정후원금"] = ((100-credits.최초감면율)*비용제외조정전합계/100).round(0)
#                 # 프리채권, 원금감면N, 계산한 최초금액이 - 인 경우 수정
# ind = credits[(credits["상환구분"]=="프리")|(credits["원금감면여부"]=="N")|(credits["접수최초조정후원금"]>credits["조정전원금"])].index
# credits.loc[ind, ["접수최초조정후원금"]] = credits["조정전원금"] # 접수후최초조정후원금 != 조정전원금 왜냐면 매입사 변제

# 1-2) 원금감면율
        # 당사 현재원금감면율은 매입전 변제금 있는 경우 정확하지 않음
credits["최초원금감면율"] = ((credits["조정전원금"]-credits["접수후최초조정후원금"])*100 / credits["조정전원금"]).round(1)
        # 현재원금감면율(기상환액포함)
credits["현재원금감면율"] = ((credits["조정전원금"] - (credits["조정후원금"]+credits["재조정이전납입금액"]))*100 / credits["조정전원금"]).round(1)
                # 프리채권, 원금감면N 수정
credits.loc[(credits["상환구분"]=="프리")|(credits["원금감면여부"]=="N"), ["현재원금감면율"]] = 0
                # 당사현재와 최초원금감면율이 다르지만, 감면율 수정된 적 없는 경우 
ind = credits[(credits["현재원금감면율"]!=credits["최초원금감면율"])
              &(credits["신용회복키"].isin(감면율로그_unique["신용회복키"])) # 감면율변화없음
              &(credits["최초합의서체결일"]<credits["컷오프일"])].index # 컷오프전 확정건(매입전 변제금이 있는 경우 현재원금감면율 계산식이 맞지 않음)
credits.loc[ind,"현재원금감면율"] = credits["최초원금감면율"]


# 2) 접수후 원금상환율 : 접수후 원금입금합계 / 접수일(또는 최초연체일+3) / 접수후기간
credits["총상환원금"] = credits["재조정이전납입금액"]+credits["총납입원금"]

credits["접수후변제기일"] = np.where(credits["접수일"] < credits["최초연체일"], credits["최초연체일"] + pd.DateOffset(months=3), credits["접수일"])
        # 매입후변제기간(월)
credits["접수후변제기간"] = (basedate_dt.year - credits["접수후변제기일"].dt.year) * 12 + (basedate_dt.month - credits["접수후변제기일"].dt.month)
                # 오류건(미확정, 미도래 등) 처리
credits.loc[(credits["접수후변제기간"]<0)|(credits["접수후변제기간"].isna()), "접수후변제기간"] = 0

        # result : 월변제율
credits["접수후월평균납입원금"] = credits["총상환원금"] // credits["접수후변제기간"] # ( / ).astype(int)와 같음, 또는 np.floor( / ).astype(int)
credits["접수후원금상환율"] = (credits["접수후월평균납입원금"]*100 / credits["접수후최초조정후원금"]).round(1)


# 3) 매입후 합계상환율 : 매입후 입금합계_6개월 / 최초조정후합계 / 매입후기간
        # 최초 변제기일 : 최초 합의서체결일 + 1개월
credits["매입후변제기일"] = np.where(credits["최초합의서체결일"]>=credits["컷오프일"], credits["최초합의서체결일"] + pd.DateOffset(months=1), credits["컷오프일"])
        # 매입후변제기간(월)
credits["매입후변제기간"] = (basedate_dt.year - credits["매입후변제기일"].dt.year) * 12 + (basedate_dt.month - credits["매입후변제기일"].dt.month)
                # 오류건(미확정, 미도래 등) 처리
credits.loc[(credits["매입후변제기간"]<0)|(credits["매입후변제기간"].isna()), "매입후변제기간"] = 0

        # result : 월변제율
credits["매입후월평균입금액"] = credits["입금합계_전기간"] // credits["매입후변제기간"]
credits["매입후합계금상환율"] = (credits["매입후월평균입금액"]*100 / credits["매입후최초상환후잔액"]).round(1)

# credits["상환율_접수최초조정후원금"] = (credits["월평균납입원금"]*100 / credits["접수최초조정후원금"]).round(1)
# credits["상환율_최초OPB"] = (credits["월평균납입원금"]*100 / credits["최초OPB"]).round(1)


# 열추가
        # 감면율등급
x = credits["현재원금감면율"]
conds = [x==0, x<=10, x<=30, x<=50, x<70, x<=100]
values = ["0", "10이하", "30이하", "50이하", "70미만", "70이상"]
credits["감면율등급"] = np.select(conds, values, default="")

        # 매입후변제기간등급
x = credits["매입후변제기간"]
conds = [x<=10, x<=40, x<=80, x<=120, x>120]
values = ["10회이하", "40회이하", "80회이하", "120회이하", "120회초과"]
credits["매입후변제기간등급"] = np.select(conds, values, default="")

        # 상환율등급 : 총상환원금 / 조정후원금, 낮을수록 안 좋음
credits["접수후원금상환율등급"] = (credits["접수후원금상환율"]*10).round(0)
credits["매입후합계금상환율등급"] = (credits["매입후합계금상환율"]*10).round(0)

credits.loc[credits["접수후원금상환율등급"]>10, "접수후원금상환율등급"] = 10
credits.loc[credits["매입후합계금상환율등급"]>10, "매입후합계금상환율등급"] = 10


        # 본회차상환액, 유예기간여부
def 본상환액찾기(row) :
    if (row["종료회차1"]=="") or pd.isna(row["종료회차1"]): # 시리즈.isna(), pd.isna(값)
        return 0, 0

    현재회차 = row["납입회차"] # + row["연체기간"] : 더하면 안됌
    회차번호=None
    for i in range(1,11) : 
        if 현재회차 <= row[f"종료회차{i}"] :
            회차번호 = i
            break
        # 회차번호 미할당
    if 회차번호 is None :
        return 0, 0
    
        # 유예기간중이면 다음 구간 변제금
    if 현재회차 <= row["유예기간"] : 
        회차번호 += 1
        
    본회차상환액 = row[f"채무조정2월상환액{회차번호}"]
    유예기간여부 = "유예기간" if 현재회차 <= row["유예기간"] else "본기간"
    return 본회차상환액, 유예기간여부

credits[["본회차상환액", "유예기간여부"]] = credits.apply(lambda row : pd.Series(본상환액찾기(row)), axis=1)
        # 최근입금등급 = n개월 입금합계 / 본회차상환액 / n : 중간에 유예면 수치가 이상할 수 있음
#######################
입금기간 = 6 # 개월수
#######################
# 최근입금등급 : 최근6개월간 월평균입금액 / 1회차상환액 : 낮을수록 안 좋음
credits["최근입금등급"] = ((credits["입금합계_6개월"] / 입금기간 / credits["본회차상환액"])*10).round(0)
credits.loc[credits["최근입금등급"]>10, "최근입금등급"] = 10

credits["반영/매입"] = (credits["반영OPB"] *100 / credits["매입가"]).round(0)
credits["80%매각시"] = credits["반영/매입"]*0.8


# 후처리
        # 타입재정의
credits[['접수후월평균납입원금', '접수후최초조정후원금', '매입가', "본회차상환액", "입금합계_6개월"]] = \
        credits[['접수후월평균납입원금', '접수후최초조정후원금', '매입가', "본회차상환액", "입금합계_6개월"]].replace([np.inf, -np.inf],0).fillna(0).astype(int)

        # 정렬
credits = credits.sort_values(by=["채무구분", "현재원금감면율", "상환구분",  "진행구분"], ascending=[False,True,False,True])


        # 제외 : 조정전원금 0, 미확정(신규)
credits["구분"] = np.where(((credits["조정전원금"] == 0)|(credits["진행구분"]=="미확정")|(credits["매입후변제기간"]==0)),"미확정등","확정")

credits_확정 = credits.query('구분=="확정"')
credits_미확정 = credits.query('구분!="확정"')


        # 필요칼럼만 남기기
cols = ["풀", "채무자키", "계좌키", "신용회복키", "채무구분", "신청인명", "신청인주민번호", "진행구분", "채권구분", "상환구분",
        "접수후원금상환율등급", "매입후합계금상환율등급", "최근입금등급", "연체기간", "재조정횟수","유예기간여부", # 매각여부 판단열
        "접수후원금상환율", "총상환원금", "접수후변제기일", "접수후최초조정후원금",
        "매입후합계금상환율", "입금합계_전기간", "매입후변제기간", "매입후최초상환후잔액",
        "감면율등급", "현재원금감면율", "최초원금감면율", "감면율","최초감면율",
        "접수일", "합의서체결일", "최초합의서체결일", "컷오프일", 
        "원금감면여부", "상각구분", "최초OPB", "반영OPB", "매입가",
        "재조정이전납입금액", "총납입원금", "총납입이자", "총납입기타채무",
        "재접수횟수", "재조정여부", "재난상환유예", "특례지원", "수정조정여부",
        "총상환기간","유예기간", "납입회차", "연체기간",
        "조정전합계", "조정전원금", "조정전이자", "조정전연체이자", "조정전비용", "조정후합계", "조정후원금", "조정후이자", "조정후연체이자", "조정후비용", 
        "채무조정2월상환액1","본회차상환액", "입금합계_6개월", "반영/매입", "80%매각시", "특별면책", "실효횟수"
        ]



functions.save_df_to_excel_underline(credits_확정[cols], join(path_base, company+"_신용회복채권등급_" + basedate +".xlsx"), key_columns_no=0)
functions.save_df_to_excel_underline(credits_미확정[cols], join(path_base, company+"_신용회복채권등급_미확정_" + basedate +".xlsx"), key_columns_no=0)

# 매입후합계금상환율등급 : 접수 후 입금 원금(매입전 포함), 접수후 기간, 낮을 수록 안 좋음
# 최근입금등급 : 입금합계_6개월, 6개월, 낮을 수록 안 좋음
# 0.특별면책 무조건 매각, 1. 매입후합계금상환율등급(3이하, 4이상), 2.최근입금등급, 3.연체기간,재조정횟수,최근입금등급,실효이력